In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import re
import json

In [25]:
from src.data_preprocessing.clean_data import process_data
from src.data_preprocessing.feature_engineering import run_feature_engineering

### Convert to CSV

In [ ]:
# New file path for the uploaded .xlsx file
xlsx_path = "raw_data/modified_scats_data_oct_2006.xlsx"

# Attempt to read the .xlsx file using openpyxl engine
excel_data = pd.read_excel(xlsx_path, engine="openpyxl")

# Save it as a CSV file
csv_path = "raw_data/modified_scats_data_oct_2006.xlsx"
# excel_data.to_csv(csv_path, index=False)

# Read the CSV file back
csv_data = pd.read_csv(csv_path)
print(csv_data.head())

### Clean data

In [ ]:
df = process_data(
    file_path="raw_data/modified_scats_data_oct_2006.csv",
    output_dir="processed_data/preprocessed_data",
    visualize_output_dir="processed_data/eda_insights",
)

### Feature engineering

In [ ]:
# Convert Date column to datetime
df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()

output_dir = "processed_data/preprocessed_data"
os.makedirs(output_dir, exist_ok=True)

seq_length = 24
test_ratio = 0.2

# Run feature engineering
processed_data = run_feature_engineering(
    df,
    seq_length=seq_length,
    test_ratio=test_ratio,
    output_dir=output_dir,
)

### Process traffic metadata

In [23]:
def extract_roads(location):
    """Extract road names from location string."""
    if not isinstance(location, str):
        return []

    # Extract base road names (without directional qualifiers)
    # First, find common patterns for road names like "WARRIGAL_RD" or "HIGH STREET_RD"
    road_patterns = re.findall(
        r"([A-Z]+(?:[ _][A-Z]+)*(?:[ _](?:RD|ST|HWY|FWY|GV|ARTERIAL|RAMPS|PARK))?)",
        location,
    )

    # Clean up road names
    cleaned_roads = []
    for road in road_patterns:
        # Skip directional prefixes if they're standalone
        if road in ["N", "S", "E", "W", "NE", "NW", "SE", "SW", "OF"]:
            continue

        # Skip if the road name contains directional information that would indicate
        # it's a full location rather than just a road name
        if re.search(r" OF | N OF | S OF | E OF | W OF |NW OF|NE OF|SW OF|SE OF", road):
            continue

        cleaned_roads.append(road)

    return cleaned_roads


def process_traffic_metadata(
    filepath, output_dir, 
    locations_to_keep=set(),
    scats_numbers_to_keep=set(),
):
    """Process traffic data to create site metadata JSON file."""
    print("=== Data Processing ===")

    # 1. Load raw data
    excel_file = filepath

    try:
        traffic = pd.read_excel(excel_file, sheet_name="Data", header=1)
        summary = pd.read_excel(excel_file, sheet_name="Summary Of Data", header=3)
        print(f"Successfully loaded Excel file: {excel_file}")
    except FileNotFoundError:
        print(f"Error: Could not find Excel file at {excel_file}")
        return
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return

    # 2. Process summary data to group locations by site ID
    site_locations = {}
    current_site_id = None

    # Cast all SCATS numbers in scats_numbers_to_keep to string for comparison
    scats_numbers_to_keep = {str(int(num)) for num in scats_numbers_to_keep}

    # Cast all locations in locations_to_keep to string for comparison
    locations_to_keep = {str(loc) for loc in locations_to_keep}

    # Iterate through rows in summary sheet
    for _, row in summary.iterrows():
        # If the row has a SCATS Number, update the current site ID
        if not pd.isna(row["SCATS Number"]):
            current_site_id = str(int(row["SCATS Number"]))

            # Initialize the site ID in the dictionary if not already present
            # Also Check if the site ID is in the list of SCATS numbers to keep
            if current_site_id not in site_locations and current_site_id in scats_numbers_to_keep:
                site_locations[current_site_id] = []

        # If we have a current site ID and the row has a Location
        if current_site_id and not pd.isna(row["Location"]):
            location = row["Location"]

            # Check if the location is in the list of locations to keep
            if isinstance(location, str) and location in locations_to_keep:
                site_locations[current_site_id].append(location)

    print(f"Found {len(site_locations)} unique SCATS sites")

    # 3. Process metadata for each site
    site_metadata = {}

    for site_id, locations in site_locations.items():
        # Extract all connected roads
        all_roads = set()
        for loc in locations:
            # Alternative approach to extract just road names
            parts = re.split(
                r"[ _]OF[ _]|[ _]of[ _]|[ _]N[ _]of[ _]|[ _]S[ _]of[ _]|[ _]E[ _]of[ _]|[ _]W[ _]of[ _]|NW OF|NE OF|SW OF|SE OF",
                loc,
            )
            for part in parts:
                if part and len(part) > 1:  # Skip empty parts or single letters
                    # Extract just the road name (removing directional indicators)
                    road_match = re.match(
                        r"([A-Z]+(?:[ _][A-Z]+)*(?:[ _](?:RD|ST|HWY|FWY|GV|ARTERIAL|RAMPS|PARK)))",
                        part.strip(),
                    )
                    if road_match:
                        road_name = road_match.group(1).strip()
                        if road_name and road_name not in [
                            "N",
                            "S",
                            "E",
                            "W",
                            "NE",
                            "NW",
                            "SE",
                            "SW",
                            "OF",
                        ]:
                            all_roads.add(road_name)

        # Get lat/lon from traffic data
        site_traffic = traffic[traffic["SCATS Number"] == int(site_id)]

        if (
            not site_traffic.empty
            and "NB_LATITUDE" in site_traffic.columns
            and "NB_LONGITUDE" in site_traffic.columns
        ):
            lat = float(site_traffic["NB_LATITUDE"].iloc[0])
            lon = float(site_traffic["NB_LONGITUDE"].iloc[0])
        else:
            lat = None
            lon = None

        # Create metadata entry
        site_metadata[site_id] = {
            "site_id": int(site_id),
            "latitude": lat,
            "longitude": lon,
            "locations": locations,  # All locations for this site
            "connected_roads": list(all_roads),
        }

        print(
            f"→ Processed Site {site_id}: {len(locations)} locations, {len(all_roads)} connected roads"
        )

    metadata_path = os.path.join(output_dir, "sites_metadata.json")
    with open(metadata_path, "w") as f:
        json.dump(site_metadata, f, indent=2)

    print(
        f"\nMetadata processing complete! Created {metadata_path} with {len(site_metadata)} sites."
    )

In [24]:
file_path = "raw_data/original_scats_data_oct_2006.xlsx"
output_dir = "processed_data/preprocessed_data"
os.makedirs(output_dir, exist_ok=True)

cleaned_df = pd.read_csv("processed_data\preprocessed_data\cleaned_data.csv")
locations_to_keep = set(cleaned_df["Location"].unique())
scats_num_to_keep = set(cleaned_df["SCATS Number"].unique())

process_traffic_metadata(
    filepath=file_path, output_dir=output_dir, 
    locations_to_keep=locations_to_keep,
    scats_numbers_to_keep=scats_num_to_keep,
)

=== Data Processing ===
Successfully loaded Excel file: raw_data/original_scats_data_oct_2006.xlsx
Found 39 unique SCATS sites
→ Processed Site 970: 4 locations, 2 connected roads
→ Processed Site 2000: 4 locations, 3 connected roads
→ Processed Site 2200: 4 locations, 2 connected roads
→ Processed Site 2820: 2 locations, 3 connected roads
→ Processed Site 2825: 1 locations, 2 connected roads
→ Processed Site 2827: 4 locations, 4 connected roads
→ Processed Site 2846: 4 locations, 2 connected roads
→ Processed Site 3001: 3 locations, 3 connected roads
→ Processed Site 3002: 4 locations, 3 connected roads
→ Processed Site 3120: 4 locations, 3 connected roads
→ Processed Site 3122: 3 locations, 2 connected roads
→ Processed Site 3126: 3 locations, 2 connected roads
→ Processed Site 3127: 3 locations, 2 connected roads
→ Processed Site 3180: 3 locations, 2 connected roads
→ Processed Site 3662: 4 locations, 4 connected roads
→ Processed Site 3682: 4 locations, 2 connected roads
→ Processe

e:\Anaconda\envs\ita_tf\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
